<a href="https://colab.research.google.com/github/royn5618/2022.pycon.sk/blob/main/Agno/Agent_Data_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal: Test an agent who can analyse and answer questions on an input CSV.

Not Completed - Producing Errors

# Imports

In [6]:
!pip install -U sqlalchemy 'psycopg[binary]' pgvector
!pip install agno

# API Keys Set-Up

In [9]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Set-Up Vector Database

In [8]:
def udocker_init():
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

# Set-Up PgVector Database

# 1. Install udocker
!udocker --allow-root install

# 2. Kill existing processes and clean up
!pkill -9 -f postgres
!rm -rf /content/pgdata
!udocker --allow-root rm pgvector
!rm -f postgres.log

# 3. Create fresh directory
!mkdir -p /content/pgdata
!chmod -R 777 /content/pgdata

# 4. Pull and create container with correct image path
!udocker --allow-root pull ankane/pgvector
!udocker --allow-root create --name=pgvector ankane/pgvector

# 5. Run the container
!nohup udocker --allow-root run \
    --env="POSTGRES_DB=ai" \
    --env="POSTGRES_USER=ai" \
    --env="POSTGRES_PASSWORD=ai" \
    --env="PGDATA=/var/lib/postgresql/data/pgdata" \
    --volume="/content/pgdata:/var/lib/postgresql/data" \
    --publish="5532:5432" \
    pgvector > postgres.log 2>&1 &

# 6. Connection testing
import time
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def test_db_connection(max_retries=5, wait_time=10):
    db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

    for attempt in range(max_retries):
        try:
            print(f"\nConnection attempt {attempt + 1}/{max_retries}")
            engine = create_engine(db_url)
            with engine.connect() as connection:
                result = connection.execute(text("SELECT version();"))
                version = result.fetchone()[0]
                print("✅ Successfully connected to PostgreSQL!")
                print(f"Server Version: {version}")

                # Test vector extension
                connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
                print("✅ Vector extension ready!")
                return engine
        except OperationalError as e:
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds...")
            print("\nChecking postgres status:")
            !ps aux | grep postgres
            print("\nLatest logs:")
            !tail -n 20 postgres.log
            time.sleep(wait_time)

    return None

# 7. Apply 30 secs sleep time to wait for DB to finish set up before testing for connection
print("Waiting for database to initialize...")
time.sleep(30)

engine = test_db_connection()

if engine:
    print("\n✅ Database is ready for RAG Agent initialization!")
else:
    print("\n❌ Database connection failed. Please check the logs above.")

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples
Error: invalid container id 
Info: downloading layer sha256:f2c967e41f72b294e2b96f25154dda38dbde3603b3be33888fb437147972f24b
Info: downloading layer sha256:c5f09b50002256f9e40253d9f3f34381edbe3ca083eb5ce77ecffc874c087995
Info: downloading layer sha256:2e3723549f1143b2c0381181709301932d6a592d8969d0827c1f0133772dfbe0
Info: downloading layer sha256:7077e54346e0cc4692391042abd0479bb02443892be7c6b1085fe7184caff826
Info: downloading layer sha256:bb153abf380255875eda2f78bb3c853520a77f3175574a91d909b5d6912c75a4
Info: downloading layer sha256:f1a157d7d7b01f004e4e758a97a38a5d10c8ce79348e5b674187a99d4f0cabda
Info: downloading layer sha256:6e662fa63f18991e2026f333e95c9670506a0c891ec82e5593bb613a627c6a96
Info: downloading layer sha256:2c35234636c95a2fed252512bb033c920753cffdd75c796da556a594845c121d
Info: downloading layer sha256:04efcdd3a2a4cbfcbdd1542bb9af0b2ff422f4e7b2cd

# Data Analyst Agent

In [13]:
from agno.agent import Agent
from agno.knowledge.csv import CSVKnowledgeBase
from agno.vectordb.pgvector import PgVector
from agno.models.openai import
from agno.embedder.google import GeminiEmbedder

db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

knowledge_base = CSVKnowledgeBase(
    path="Data/airlines_flights_data.csv",
    vector_db=PgVector(
        table_name="csv_documents",
        db_url=db_url,
        embedder=GeminiEmbedder(),
    ),
    num_documents=5,  # Number of documents to return on search
)


In [14]:
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Data/airlines_flights_data.csv

INFO Inserted batch of 1 documents.

INFO Inserted batch of 1 documents.

...

ERROR    Error processing document 'airlines_flights_data': 429 RESOURCE_EXHAUSTED. {'error': {'code': 429,        
         'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

/usr/local/lib/python3.11/dist-packages/agno/vectordb/pgvector/pgvector.py:338: SAWarning: Column 'ai.csv_documents.id' is marked as a member of the primary key for table 'ai.csv_documents', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL.
  sess.execute(insert_stmt, batch_records)


ERROR    Error with batch starting at index 0: (psycopg.errors.NotNullViolation) null value in column "id" of      
         relation "csv_documents" violates not-null constraint                                                     
         DETAIL:  Failing row contains (null, null, {}, {}, null, null, null, 2025-08-10 11:18:45.524761+00, null, 
         null).                                                                                                    
         [SQL: INSERT INTO ai.csv_documents DEFAULT VALUES]                                                        
         (Background on this error at: https://sqlalche.me/e/20/gkpj)

ERROR    Error inserting documents: (psycopg.errors.NotNullViolation) null value in column "id" of relation        
         "csv_documents" violates not-null constraint                                                              
         DETAIL:  Failing row contains (null, null, {}, {}, null, null, null, 2025-08-10 11:18:45.524761+00, null, 
         null).                                                                                                    
         [SQL: INSERT INTO ai.csv_documents DEFAULT VALUES]                                                        
         (Background on this error at: https://sqlalche.me/e/20/gkpj)

IntegrityError: (psycopg.errors.NotNullViolation) null value in column "id" of relation "csv_documents" violates not-null constraint
DETAIL:  Failing row contains (null, null, {}, {}, null, null, null, 2025-08-10 11:18:45.524761+00, null, null).
[SQL: INSERT INTO ai.csv_documents DEFAULT VALUES]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [15]:
# Initialize the Agent with the knowledge_base
agent = Agent(
    model=Gemini("gemini-1.5-flash"),
    knowledge=knowledge_base,
    search_knowledge=True,
)

Output()

INFO Found 5 documents